In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Importing IA libraries
from qiskit.circuit import Parameter, ParameterVector
from qiskit.algorithms.optimizers import GradientDescent
from qiskit_machine_learning.neural_networks import OpflowQNN

# Loading IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
from qiskit_machine_learning.datasets import ad_hoc_data

training_features, training_labels, test_features, test_labels = ad_hoc_data(
    training_size=20, test_size=10, n=2, one_hot=False, gap=0.5
)

# the training labels are in {0, 1} but we'll use {-1, 1} as class labels!
training_labels = 2 * training_labels - 1
test_labels = 2 * test_labels - 1

In [3]:
from qiskit.utils import QuantumInstance

backend = Aer.get_backend('qasm_simulator')
q_instance = QuantumInstance(backend, shots = 8192, seed_simulator = 2718, seed_transpiler = 2718)

In [4]:
from qiskit.circuit.library import ZZFeatureMap

dim = 2
feature_map = ZZFeatureMap(dim, reps=1)

In [5]:
from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(num_qubits=dim, entanglement='linear', reps=1)  # also simple here!
circuit = feature_map.compose(ansatz)

In [6]:
gd_qnn_loss = []
def gd_qnn_callback(*args):
    gd_qnn_loss.append(args[2])
    
gd = GradientDescent(maxiter = 100, learning_rate = 0.01, callback = gd_qnn_callback)

In [7]:
from qiskit.opflow import StateFn, Z, PauliExpectation, Gradient

hamiltonian = Z ^ Z

qnn_expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(circuit)

qnn = OpflowQNN(qnn_expectation, 
                input_params = list(feature_map.parameters), 
                weight_params = list(ansatz.parameters),
                exp_val = PauliExpectation(),
                gradient = Gradient(),  # <-- Parameter-Shift gradients
                quantum_instance = q_instance)

In [8]:
from qiskit_machine_learning.algorithms import NeuralNetworkClassifier
#initial_point = np.array([0.2, 0.1, 0.3, 0.4])
classifier = NeuralNetworkClassifier(qnn, optimizer=gd)

In [ ]:
classifier.fit(training_features, training_labels);

In [ ]:
predicted = classifier.predict(test_features)